# Flag regions

AMRClaw and GeoClaw version 5.6.1 (and earlier) allow specifying rectangular refinement regions (see [www.clawpack.org/regions.html](http://www.clawpack.org/regions.html)) in `setrun.py` in the form of a list that is appended to `rundata.regiondata.regions`:

```
rundata.regiondata.regions.append([minlevel,maxlevel,t1,t2,x1,x2,y1,y2])
```
This is a region that is active from time `t1` to `t2` over the spatial extent `[x1,x2,y1,y2]`.

The modified version of GeoClaw supports a new approach to specifying regions that I propose calling `flagregions`.  This will also support simple rectangles and so should ultimately replace `regions` in both AMRClaw and GeoClaw.  The name `flagregions` is more descriptive of the function served by these regions and may cause less confusion with other uses of the word "region" in various applications.

**NOTE:** The class `FlagRegion` discussed below is currently implemented in a separate module `data_FlagRegions.py` but should eventually be moved to `clawpack.amrclaw.data`?

The new way of specifying a flag region in `setrun.py` is to first define an object `flagregion` of class `FlagRegion`, set various attributes of this object (including `minlevel`, `maxlevel`, `t1`, `t2`, and a spatial extent), and then append this object to `rundata.flagregiondata.flagregions`.  Here is an example:

```
flagregion = data_FlagRegions.FlagRegion(num_dim=2)
flagregion.name = 'Region_AdmiraltyInlet'
flagregion.minlevel = 4
flagregion.maxlevel = 5
flagregion.t1 = 3600.
flagregion.t2 = 1e9
flagregion.spatial_region_file = os.path.abspath('RuledRectangle_AdmiraltyInlet.data')
rundata.flagregiondata.flagregions.append(flagregion)
```

Note that the spatial extent in this case is specified by a path to a file that describes a Ruled Rectangle, in this case one that covers Admiralty Inlet as defined in one of the examples of  [RuledRectangles.ipynb](RuledRectangles.ipynb).

So far this is the only way to define the spatial region, but note that a simple rectangle can be specified easily via, e.g.:

```
# Define and write a Ruled Rectangle corresponding to the 
# rectangle with extent [x1,x2,y1,y2]:
from clawpack.amrclaw.region_tools import RuledRectangle
rectangle1 = RuledRectangle(rect=[x1,x2,y1,y2])
rectangle1.write('RuledRectangle1.data')

# Use this RuledRectangle as the spatial extent in a flagregion:
flagregion = data_FlagRegions.FlagRegion(num_dim=2)
flagregion.name = 'Region_Rectangle1'
# set minlevel, maxlevel, t1, t2
flagregion.spatial_region_file = os.path.abspath('RuledRectangle1.data')
rundata.flagregiondata.flagregions.append(flagregion)
```

After defining `rundata` in `setrun.py` this data is then written out to a new data file `flagregions.data`, which is read in by the GeoClaw Fortran code.

## Possible future enhancements

### Other spatial regions

Perhaps allow setting 
```
flagregion.spatial_region = [x1,x2,y1,y2]
```
directly for a simple rectangle, and/or 
```
flagregion.spatial_region = rr
```
where `rr` is an object of class `RuledRectangle`.  Note that this will require writing something different to `flagregions.data` and then reading it properly from the Fortran code, and has not yet been implemented.

### Other attributes

In the future we may implement the ability to specify different tolerances or even different flagging methods in each region.  If so, other attributes such as `flagregion.tolerance` or `flagregion.method` could be easily added.

[Back to Index.ipynb](Index.ipynb)